# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [29]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine

import pickle
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer


from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier as clf
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

nltk.download(['punkt','wordnet','stopwords', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# load data from database

engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql("SELECT * FROM InsertTableName", engine)
#df.genre.value_counts()

#df.head()
X = df.message.values
y = df.iloc[:,4:]


### 2. Write a tokenization function to process your text data

In [56]:
# Checking the messages
y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_regex, " ", text.lower().strip()) 
    words = word_tokenize(text)
    tokens = [WordNetLemmatizer().lemmatize(w) for w in words if w not in stopwords.words("english")]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('clf',clf(RandomForestClassifier()))
    
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [20]:
# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)
# Fitting the training set to the pipeline
pipeline.fit(X_train,y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001D2F58DB040>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [22]:
# Predicting on the test set
y_pred = pipeline.predict(X_test)

# printing the classification report
for i in range(len(y.axes[1])):
    print(y_test.columns[i], '-')
    
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

related -
              precision    recall  f1-score   support

           0       0.69      0.43      0.53      1456
           1       0.85      0.94      0.89      5023
           2       0.26      0.22      0.24        49

    accuracy                           0.82      6528
   macro avg       0.60      0.53      0.55      6528
weighted avg       0.81      0.82      0.80      6528

request -
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5431
           1       0.85      0.49      0.62      1097

    accuracy                           0.90      6528
   macro avg       0.88      0.74      0.78      6528
weighted avg       0.90      0.90      0.89      6528

offer -
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6503
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6528
   macro avg       0.50      0.50      0.50   

C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6126
           1       0.89      0.30      0.45       402

    accuracy                           0.95      6528
   macro avg       0.92      0.65      0.71      6528
weighted avg       0.95      0.95      0.94      6528

food -
              precision    recall  f1-score   support

           0       0.95      0.98      0.97      5777
           1       0.83      0.59      0.69       751

    accuracy                           0.94      6528
   macro avg       0.89      0.79      0.83      6528
weighted avg       0.93      0.94      0.93      6528

shelter -
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      5949
           1       0.84      0.40      0.54       579

    accuracy                           0.94      6528
   macro avg       0.89      0.69      0.75      6528
weighted avg       0.93      0.94      0.93      6528

cl

C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C

              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5930
           1       0.78      0.50      0.61       598

    accuracy                           0.94      6528
   macro avg       0.87      0.74      0.79      6528
weighted avg       0.94      0.94      0.94      6528

fire -
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6440
           1       1.00      0.01      0.02        88

    accuracy                           0.99      6528
   macro avg       0.99      0.51      0.51      6528
weighted avg       0.99      0.99      0.98      6528

earthquake -
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      5906
           1       0.88      0.80      0.84       622

    accuracy                           0.97      6528
   macro avg       0.93      0.89      0.91      6528
weighted avg       0.97      0.97      0.97      6528


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {'clf__estimator__n_estimators':[10,50],
              'clf__estimator__min_samples_split':[2,3]
             }

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)
'''Note: Search reduced to a smaller range since the time to run becomes extremely high'''

In [26]:
# printing the best parameters
print("\nBest Parameters:", cv.best_params_)
# Setting the model parameters to the best parameters
model = cv.best_estimator_


Best Parameters: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 50}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred= model.predict(X_test)

In [32]:
# printing the classification report
for i in range(len(y.axes[1])):
    print(y_test.columns[i], '-')
    
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))#, target_names=y.columns))

related -
              precision    recall  f1-score   support

           0       0.69      0.43      0.53      1456
           1       0.85      0.94      0.89      5023
           2       0.29      0.18      0.23        49

    accuracy                           0.82      6528
   macro avg       0.61      0.52      0.55      6528
weighted avg       0.81      0.82      0.81      6528

request -
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5431
           1       0.83      0.50      0.63      1097

    accuracy                           0.90      6528
   macro avg       0.87      0.74      0.78      6528
weighted avg       0.89      0.90      0.89      6528

offer -
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6503
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6528
   macro avg       0.50      0.50      0.50   

C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6324
           1       0.62      0.06      0.12       204

    accuracy                           0.97      6528
   macro avg       0.79      0.53      0.55      6528
weighted avg       0.96      0.97      0.96      6528

child_alone -
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528

    accuracy                           1.00      6528
   macro avg       1.00      1.00      1.00      6528
weighted avg       1.00      1.00      1.00      6528

water -
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6126
           1       0.89      0.29      0.44       402

    accuracy                           0.95      6528
   macro avg       0.92      0.65      0.71      6528
weighted avg       0.95      0.95      0.94      6528

food -
              precision    recall  f1-score 

C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kumar\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6448
           1       0.00      0.00      0.00        80

    accuracy                           0.99      6528
   macro avg       0.49      0.50      0.50      6528
weighted avg       0.98      0.99      0.98      6528

other_infrastructure -
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6237
           1       0.00      0.00      0.00       291

    accuracy                           0.96      6528
   macro avg       0.48      0.50      0.49      6528
weighted avg       0.91      0.96      0.93      6528

weather_related -
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      4679
           1       0.85      0.70      0.77      1849

    accuracy                           0.88      6528
   macro avg       0.87      0.83      0.84      6528
weighted avg       0.88      0.88

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [37]:
 print(model)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001D2F58DB040>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=50)))])


### 9. Export your model as a pickle file

In [36]:
#with open('classifier.pkl','wb') as file:
    #pickle.dump(cv, file)
    
pickle.dump(model, open('cv.pkl', 'wb'))

PicklingError: Can't pickle <function tokenize at 0x000001D2F58DB040>: it's not the same object as __main__.tokenize

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.